In [1]:
import tensorflow as tf
import numpy as np
import cv2
import os

import matplotlib.pyplot as plt 
%matplotlib inline

# Model : VGG16

In [2]:
def pRelu(x, name='P_ReLU'):
    with tf.variable_scope(name):
        alpha = tf.get_variable('a', x.get_shape()[-1], initializer=tf.constant_initializer(0.0), dtype=tf.float32)
        return tf.maximum(0.0, x) + tf.minimum(0.0, alpha*x)

In [3]:
def conv2d(input_, output_n, k_h=3, k_w=3, d_h=1, d_w=1, bias=0.0, activation_fc=pRelu, name='conv2d'):
    with tf.variable_scope(name):
        W = tf.get_variable('W', [k_h, k_w, input_.get_shape()[-1], output_n],
                            initializer=tf.contrib.layers.xavier_initializer())
        b = tf.get_variable('b', [output_n], initializer=tf.constant_initializer(bias))
        conv = tf.nn.conv2d(input_, W, strides=[1, d_h, d_w, 1], padding='SAME', name='conv')
        conv_b = tf.nn.bias_add(conv, b)
        act = activation_fc(conv_b, 'act')
        
        tf.summary.histogram('Weight', W)
        tf.summary.histogram('Bias', b)
        tf.summary.histogram('Conv', conv)
        tf.summary.histogram('Conv_with_bias', conv_b)
        tf.summary.histogram('Activation', act)

        return act


def flatten(input_, name='flatten'):
    vec_dim = input_.get_shape()[1:]
    n = vec_dim.num_elements()
    with tf.name_scope(name):
        return tf.reshape(input_, [-1, n])


def linear(input_, output_size, stddev=0.02, bias=0.0, name='linear'):
    input_size = input_.get_shape().as_list()[1]
    with tf.variable_scope(name):
        W = tf.get_variable('W', [input_size, output_size], tf.float32,
                           initializer=tf.random_normal_initializer(stddev=stddev))
        b = tf.get_variable('b', [output_size], initializer=tf.constant_initializer(bias))
        logits = tf.nn.xw_plus_b(input_, W, b, name='logits')

        tf.summary.histogram('Weight', W)
        tf.summary.histogram('Bias', b)
        tf.summary.histogram('logit', logits)

        return logits
    
def drop_out(x, prob, name='drop_out'):
    with tf.variable_scope(name):
        drop_layer = tf.nn.dropout(x, prob)
        tf.summary.histogram('Drop_Out', drop_layer)
        return drop_layer

def wsl_cross_enropy(cls_pred, loc_pred, true, a=0.1):
    '''
    :param a: Hyperparameter. default = 0.1 and increased to 0.9 after 60 epochs
             to determine the level of importance between classifier and localizer.
    '''
    cls_softmax = tf.nn.softmax(cls_pred, name='Clssification_Softmax')
    loc_softmax = tf.nn.softmax(loc_pred, name='Localization_Softmax')

    return - (1 - a)*tf.reduce_mean(true * tf.log(cls_softmax)) - a * tf.reduce_mean(true * tf.log(loc_softmax))

def batch_norm(x, epsilon=1e-5, momentum=0.9, train=True, name="batch_norm"):
    with tf.variable_scope(name):
        return tf.contrib.layers.batch_norm(x, decay=momentum,
                                            updates_collections=None, epsilon=epsilon, scale=True, scope=name)

In [4]:
class STL(object):
    def __init__(self):
        self.output_n = 2
        self.class_n = 2
        self.default_act_fn = pRelu
        print("Self-Transfer-Learning is Ready")
    
    def conv_layer(self, input_):
        self.conv1_1 = conv2d(input_, output_n=96, k_h=11, k_w=11, d_h=4, d_w=4, bias=0.0, activation_fc=self.default_act_fn, name='Conv1')
        self.pool1 = tf.nn.max_pool(self.conv1_1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME', name='Pool1')
        self.norm1 = batch_norm(self.pool1, epsilon=1e-5, momentum=0.9, train=True, name="Batch_Norm1")
        
        self.conv2 = conv2d(self.norm1, output_n=256, k_h=5, k_w=5, d_h=1, d_w=1, bias=0.0, activation_fc=self.default_act_fn, name='Conv2')
        self.pool2 = tf.nn.max_pool(self.conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME', name='Pool2')
        self.norm2 = batch_norm(self.pool2, epsilon=1e-5, momentum=0.9, train=True, name="Batch_Norm2")
        
        self.conv3 = conv2d(self.norm2, output_n=384, k_h=3, k_w=3, d_h=1, d_w=1, bias=0.0, activation_fc=self.default_act_fn, name='Conv3')

        self.conv4 = conv2d(self.conv3, output_n=384, k_h=3, k_w=3, d_h=1, d_w=1, bias=0.0, activation_fc=self.default_act_fn, name='Conv4')
        
        self.conv5 = conv2d(self.conv4, output_n=256, k_h=3, k_w=3, d_h=1, d_w=1, bias=0.0, activation_fc=self.default_act_fn, name='Conv5')
        self.pool5 = tf.nn.max_pool(self.conv5, ksize=[1,3,3,1], strides=[1,2,2,1], padding='VALID', name='Pool3')
        
        return self.pool5
        
        
    def classifier(self, input_, keepratio):
        self.flat1 = flatten(input_, name='flatten')
        self.line1 = linear(self.flat1, 4096, name='fc_layer_4096')
        self.drop2 = drop_out(self.line1, prob=keepratio)
        self.line2 = linear(self.drop2, 4096, name='fc_layer_1000')
        self.drop2 = drop_out(self.line2, prob=keepratio)
        self.line3 = linear(self.drop2, self.class_n, name='fc_layer_'+str(self.class_n))
        return self.line3
    
    def localizer(self, input_, alpha):
        # Weight : Gaussian dist with standard deviation 0.01, and biases = 0
        self.conv6 = conv2d(input_, output_n = 2048, k_h=1, k_w=1, d_h=1, d_w=1, bias=0.0, activation_fc=self.default_act_fn, name='Conv6')
        print(self.conv6)
        self.gap = tf.reduce_mean( self.conv6, [1,2] )
        with tf.variable_scope("GAP"):
            self.gap_w = tf.get_variable("GAP_W", shape=[2048, self.class_n],initializer=tf.random_normal_initializer(0., 0.01))
            print(self.gap_w)

        return tf.matmul(self.gap, self.gap_w)


    def get_classmap(self, conv6, label):
        conv6_resized = tf.image.resize_bilinear( conv6, [224, 224] )
        with tf.variable_scope("GAP", reuse=True):
            label_w = tf.gather(tf.transpose(tf.get_variable("W")), label)
            print(label_w)
            label_w = tf.reshape( label_w, [-1, 2048, 1] )
            print(label_w)

        conv6_resized = tf.reshape(conv6_resized, [-1, 224*224, 2048]) 

        classmap = tf.batch_matmul( conv6_resized, label_w )
        classmap = tf.reshape( classmap, [-1, 224,224] )
        return classmap

    
    def inference(self, input_, keepratio, alpha):
        conv_layers = self.conv_layer(input_)
        cls_pred = self.classifier(conv_layers, keepratio)
        loc_pred = self.localizer(conv_layers, alpha)
        return cls_pred, loc_pred
    

In [5]:
img_size = 500
channel_n = 1
class_n = 2

In [6]:
x = tf.placeholder(tf.float32, shape=[None, img_size, img_size, channel_n])
y = tf.placeholder(tf.float32, shape=[None, class_n])
alpha = tf.placeholder(tf.float32)
keepratio = tf.placeholder(tf.float32) 

In [7]:
model = STL()

Self-Transfer-Learning is Ready


In [8]:
cls_pred, loc_pred = model.inference(x, keepratio, alpha)

Tensor("Conv6/act/add:0", shape=(?, 15, 15, 2048), dtype=float32)
Tensor("GAP/GAP_W/read:0", shape=(2048, 2), dtype=float32)


In [9]:
cost = wsl_cross_enropy(cls_pred, loc_pred, y, alpha)   # 처음엔 0.1로 시작하고서, 60 epochs 후에는 0.9

# Image Ready

#### 